# Displaying Process Models

Here we can display a discovered process model. The event log must match the event log the process model is based on.

In [1]:
import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))

from ipywidgets import widgets, interact, interact_manual, Layout, Button, Box
from IPython.display import display

from april.fs import EVALUATION_DIR, EVENTLOG_DIR
from april.utils import microsoft_colors
from april import Dataset
from april.processmining import EventLog

import matplotlib.pyplot as plt
import pygraphviz
import networkx as nx

from networkx.algorithms.dag import dag_longest_path_length
from networkx.algorithms.traversal.breadth_first_search import generic_bfs_edges

import os

parameters_gui = widgets.Output()
output = widgets.Output()

event_log_widget = widgets.Dropdown(description='Event Log')
graph_widget = widgets.Dropdown(description='Process Model')

show_button = widgets.Button(description="Show")
reload_button = widgets.Button(description="Reload")

with parameters_gui:
    display(widgets.VBox([event_log_widget, graph_widget, widgets.HBox([show_button, reload_button])]))

def get_all_event_logs():
    files = os.listdir(EVENTLOG_DIR)
    return [file[0:len(file)-8] for file in files if file[-8:] == '.json.gz']

def get_all_graphs():
    files = os.listdir(EVALUATION_DIR)
    return [file[0:len(file)-8] for file in files if file[-8:] == '.gpickle']

def on_reload(button):
    event_log_widget.options = get_all_event_logs()
    graph_widget.options = get_all_graphs()

def graph_add_display_names(graph, coder_attributes):
    """ Add display names to a graph.
        :param graph: The graph for which the display names are to be added.
        :param coder_attributes: Decoding labels to display name. """
    # add identifier attribute
    for node in graph.nodes:
        node_attributes = graph.nodes[node]
        identifier = coder_attributes.decode(node_attributes['label'], node_attributes['attribute'])
        node_attributes['display_name'] = identifier

def show_process_model(button):      # TODO: use event attributes
    dataset = Dataset(event_log_widget.value, use_event_attributes=True)
    coder_attributes=dataset.get_encoder_decoder_for_attributes()
    
    graph = nx.read_gpickle(f"{EVALUATION_DIR / graph_widget.value}.gpickle")
    
    length = dag_longest_path_length(graph)

    root = [n for n, d in graph.in_degree() if d == 0][0]
    graph_width = max(len(nx.descendants_at_distance(graph, root, d)) for d in range(length))
    
    width = graph_width * 3
    height = length * 1.5
    
    fig = plt.figure(1, (width, height))

    pos = nx.drawing.nx_agraph.graphviz_layout(graph, prog='dot')
    edge_labels = nx.get_edge_attributes(graph, 'probability')
    node_labels = nx.get_node_attributes(graph, 'label')


    # add display names if given encoder and decoder
    if coder_attributes is not None:
        graph_add_display_names(graph, coder_attributes)
        node_labels = nx.get_node_attributes(graph, 'display_name')

    attribute_colors = microsoft_colors[3:]
    colors = dict(zip(range(dataset.num_attributes-1), attribute_colors))

    color_map = []
    for node in graph:
        if graph.nodes[node]['display_name'] in [str(EventLog.start_symbol), str(EventLog.end_symbol)]:
            color_map.append(microsoft_colors[0])
        elif graph.nodes[node]['attribute'] == 0:
            color_map.append(microsoft_colors[2])
        else:
            color_map.append(colors[graph.nodes[node]['attribute']-1])

    nx.draw_networkx_nodes(graph, pos, node_color=color_map)  
    nx.draw_networkx_edges(graph, pos)
    nx.draw_networkx_edge_labels(graph, pos, edge_labels=edge_labels)
    nx.draw_networkx_labels(graph, pos, labels=node_labels)
    
    output.clear_output()
    with output:
        plt.show()

show_button.on_click(show_process_model)

reload_button.on_click(on_reload)
on_reload(None)

display(parameters_gui)
display(output)

Output()

Output()